<a href="https://colab.research.google.com/github/desstaw/DataPrivacy_SimulatedAnnealing/blob/main/Main_K_Anonymity_Simulated_Annealing_Balancing_Stroke_MIMICIII.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import the data and briefly explore it again

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, roc_curve
import matplotlib.pyplot as plt

import random
import math

In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 2.2 MB/s eta 0:00:00


In [ ]:
url = "/content/drive/MyDrive/Colab Notebooks/MIMIC III/preprocessed/pre-processed-MIMICIII-SA(arx_gen).csv"
df = pd.read_csv(url)

In [ ]:
'''
df.rename(columns={'admission_type': 'L2_Admission_Type',
                   'diseases': 'L3_Diseases',
                   'ethnicity': 'L4_Ethnicity'}, inplace=True)
'''

"\ndf.rename(columns={'admission_type': 'L2_Admission_Type',\n                   'diseases': 'L3_Diseases',\n                   'ethnicity': 'L4_Ethnicity'}, inplace=True)\n"

test k-anon before

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2655 entries, 0 to 2654
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   L7_age             2655 non-null   float64
 1   gender             2655 non-null   int64  
 2   L3_Ethnicity       2655 non-null   object 
 3   marital_status     2655 non-null   object 
 4   infarct_type       2655 non-null   object 
 5   anion_gap          2655 non-null   float64
 6   abp_diastolic      2655 non-null   float64
 7   abp_systolic       2655 non-null   float64
 8   bicarbonate        2500 non-null   float64
 9   calcium            2655 non-null   float64
 10  chloride           2655 non-null   float64
 11  creatinine         2655 non-null   float64
 12  gcs                2635 non-null   float64
 13  glucose            2655 non-null   float64
 14  heart_rate         2655 non-null   float64
 15  hemoglobin         2655 non-null   float64
 16  oasis              2655 

## Necessary pre-processing for CatBoost

In [ ]:
df ['gender'] = df ['gender'].astype(str)
#df ['ethnicity'] = df ['ethnicity'].astype(str)
df ['marital_status'] = df ['marital_status'].astype(str)
df ['infarct_type'] = df ['infarct_type'].astype(str)
df ['stroke_type'] = df ['stroke_type'].astype(str)
#df ['admission_type'] = df ['admission_type'].astype(str)
df ['death_365_days'] = df ['death_365_days'].astype(str)
'''
df ['Level_1_Age'] = df ['Level_1_Age'].astype(str)
df ['Level_2_Age'] = df ['Level_2_Age'].astype(str)
df ['Level_3_Age'] = df ['Level_3_Age'].astype(str)
df ['Level_4_Age'] = df ['Level_4_Age'].astype(str)
df ['Level_5_Age'] = df ['Level_5_Age'].astype(str)
df ['Level_6_Age'] = df ['Level_6_Age'].astype(str)


df ['Level_1_los_hours'] = df ['Level_1_los_hours'].astype(str)
df ['Level_2_los_hours'] = df ['Level_2_los_hours'].astype(str)
df ['Level_3_los_hours'] = df ['Level_3_los_hours'].astype(str)
df ['Level_4_los_hours'] = df ['Level_4_los_hours'].astype(str)
df ['Level_5_los_hours'] = df ['Level_5_los_hours'].astype(str)
df ['Level_6_los_hours'] = df ['Level_6_los_hours'].astype(str)
'''
df ['L1_Admission_Type'] = df ['L1_Admission_Type'].astype(str)
df ['L2_Admission_Type'] = df ['L2_Admission_Type'].astype(str)

df ['L1_Ethnicity'] = df ['L1_Ethnicity'].astype(str)
df ['L2_Ethnicity'] = df ['L2_Ethnicity'].astype(str)
df ['L3_Ethnicity'] = df ['L3_Ethnicity'].astype(str)


df ['L1_Diseases'] = df ['L1_Diseases'].astype(str)
df ['L2_Diseases'] = df ['L2_Diseases'].astype(str)

In [ ]:
#Since the catboost model cannot read columns with interval values which are originally intended to be categories in the form of interval strings
def generate_interval_mappings(df, interval_columns):
    interval_mapping = {}

    for column in interval_columns:
        unique_intervals = df[column].unique()
        for interval in unique_intervals:
            if isinstance(interval, tuple):  # Check if the interval is a tuple
                mapping_key = f"{column}_{interval[0]}_{interval[1]}"
                interval_mapping[interval] = mapping_key
                print('interval mapping', interval_mapping)

    return interval_mapping

# List of columns with interval values
interval_columns = df[['L1_age',
       'L2_age', 'L3_age', 'L4_age', 'L5_age',
       'L1_age', 'L1_los_hours', 'L2_los_hours',
       'L3_los_hours', 'L4_los_hours', 'L5_los_hours',
       'L6_los_hours']]

# Generate interval mappings
interval_mapping = generate_interval_mappings(df, interval_columns)

# Apply the interval mapping to create new categorical columns
for column, mapping in interval_mapping.items():
    df[mapping] = df[column].apply(lambda interval: mapping if interval in interval_mapping else None)

# Drop the original columns with intervals
for column in interval_mapping.keys():
    df.drop(column, axis=1, inplace=True)


## K-anonymity and Simulated Annealing

### Quasi identiifers are formed as key: set. The aim to swap from each corresponding set later and not randomly from the whole columns.

In [ ]:
# Define the QI columns and their possible values
quasi_identifiers = {
    'age': [ 'L1_age', 'L2_age', 'L3_age', 'L4_age', 'L5_age', 'L6_age', 'L7_age'],
    'los_hours': ['L1_los_hours ', 'L2_los_hours ', 'L3_los_hours ', 'L4_los_hours ', 'L5_los_hours', 'L6_los_hours', 'L7_los_hours'],
    'admission_type': ['L2_Admission_Type', 'L1_Admission_Type'],
    'gender': ['gender'],
    'stroke_type': ['stroke_type'],
    'marital_status':['marital_status'],
    'diseases': ['L1_Diseases', 'L2_Diseases'],
    'ethnicity': ['L1_Ethnicity', 'L2_Ethnicity', 'L3_Ethnicity']
}

# Define the initial solution
initial_solution = [
    random.choice(quasi_identifiers['age']),
    random.choice(quasi_identifiers['los_hours']),
    random.choice(quasi_identifiers['admission_type']),
    random.choice(quasi_identifiers['gender']),
    random.choice(quasi_identifiers['stroke_type']),
    random.choice(quasi_identifiers['marital_status']),
    random.choice(quasi_identifiers['diseases']),
    random.choice(quasi_identifiers['ethnicity']),

]

print(initial_solution)

initial_solution = ['L3_age', 'L1_los_hours', 'L1_Admission_Type', 'gender', 'stroke_type', 'marital_status', 'L2_Diseases', 'L3_Ethnicity']

['L6_age', 'L2_los_hours ', 'L2_Admission_Type', 'gender', 'stroke_type', 'marital_status', 'L1_Diseases', 'L2_Ethnicity']


###Simulated annealing parameters

In [ ]:
# temperature and cooling rate for SA
initial_temperature = 1000 #250.0 #1000
cooling_rate = 0.01
min_temperature = 0.001

# number of iterations at each temperature level
num_iterations =  100

# maximum number of iterations without improvement
max_iterations_without_improvement = 40

total_iterations = 1000  # Initialize the total number of iterations


# Calculate the number of temperature levels based on total_iterations and num_iterations
num_temperature_levels = total_iterations // num_iterations

# Use num_temperature_levels to set the range of the temperature loop
current_temperature = initial_temperature

# Adjust the current temperature using the cooling rate
for _ in range(num_temperature_levels):
    current_temperature *= cooling_rate

### Classification function

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score

def calculate_classification_accuracy(selected_columns, df, k):
    # Prepare the feature matrix (X) and target variable (y)
    additional_columns =  ['infarct_type', 'anion_gap', 'abp_diastolic',
       'abp_systolic', 'bicarbonate', 'calcium', 'chloride', 'creatinine',
        'gcs', 'glucose', 'heart_rate', 'hemoglobin',  'oasis', 'wbc',
       'platelet_count', 'potassium', 'repiratory_rate', 'sodium']

    X = df[selected_columns + additional_columns]
    y = df['death_365_days']

    # Print the number of rows in X and y
    print('X columns:', X.columns)
    print("Number of rows in X:", len(X))
    print("Number of rows in y:", len(y))

    # Convert categorical features to strings
    categorical_features_indices = X.select_dtypes(include=['object']).columns

    random_seed = 32
    np.random.seed(random_seed)

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)

    # Initialize the CatBoostClassifier
    catboost_model = CatBoostClassifier(iterations=150, depth=7, learning_rate=0.08, loss_function='Logloss', cat_features=list(categorical_features_indices))

    # Train the model on the training data
    catboost_model.fit(X_train, y_train, cat_features=list(categorical_features_indices))

    # Predict on the testing data
    y_pred = catboost_model.predict(X_test)

    # Calculate and return the classification accuracy
    accuracy = accuracy_score(y_test, y_pred)

    # Calculate the AUC-ROC score
    auc_roc = roc_auc_score(y_test, y_pred)
    print("AUC-ROC Score:", auc_roc)

    return auc_roc


###Objective Function

In [ ]:
# k-anonymity threshold
k = 3

def objective_function(solution, max_suppressed_fraction):
    # Group the data based on the solution
    grouped = df.groupby(solution)
    total_rows_count = 0

    additional_columns =  ['infarct_type', 'anion_gap', 'abp_diastolic',
       'abp_systolic', 'bicarbonate', 'calcium', 'chloride', 'creatinine',
        'gcs', 'glucose', 'heart_rate',
       'hemoglobin',  'oasis', 'wbc',
       'platelet_count', 'potassium', 'repiratory_rate', 'sodium',
       'death_365_days']



    df_tmp = df[solution + additional_columns]

    # Calculate the percentage of suppressed rows -- utility
    total_rows = len(df)
    suppressed_rows = sum(len(group) for group_name, group in grouped if len(group) < k)
    suppressed_fraction = suppressed_rows / total_rows
    print('SUPPRESSION FRACTION', suppressed_fraction)
    #print('Total Suppressed Rows:', suppressed_rows)
    print('Max Suppressed Fraction:', max_suppressed_fraction)


    suppressed_fraction = round(suppressed_fraction, 2)
    if suppressed_fraction <= max_suppressed_fraction: #+ 1e-6:
    #if testx < testy :

        # Filter the original dataframe to remove suppressed rows
        filtered_df = df_tmp[~df_tmp.index.isin([idx for group_name, group in grouped if len(group) < k for idx in group.index])]
        print('FILTERED DF INFO', filtered_df.info())

        # Calculate and return classification accuracy as the objective function score
        classification_accuracy = calculate_classification_accuracy(solution, filtered_df, k)
        print('Classification accuracy (AUC-ROC)', classification_accuracy)
        return classification_accuracy

    else: #suppressed_fraction > max_suppressed_fraction:
        # If the constraint is violated, return a penalty (negative infinity)
        print('SUPPRESSION EXCEEDED LIMIT')
        return -float('inf')


'''
      ['weight', 'infarct_type', 'hypertension_flag', 'diabetes_flag', 'cancer_flag',
       'obesity_flag', 'drug_abuse_flag', 'anion_gap', 'abp_diastolic',
       'abp_systolic', 'bicarbonate', 'calcium', 'chloride', 'creatinine',
       'gauges_total', 'gcs', 'glucose', 'heart_rate',
       'hemoglobin', 'o2_saturation', 'oasis', 'packed_rbc',
       'platelet_count', 'potassium', 'repiratory_rate', 'sodium',
       'wbc', 'death_365_days']
'''

"\n      ['weight', 'infarct_type', 'hypertension_flag', 'diabetes_flag', 'cancer_flag',\n       'obesity_flag', 'drug_abuse_flag', 'anion_gap', 'abp_diastolic',\n       'abp_systolic', 'bicarbonate', 'calcium', 'chloride', 'creatinine',\n       'gauges_total', 'gcs', 'glucose', 'heart_rate',\n       'hemoglobin', 'o2_saturation', 'oasis', 'packed_rbc',\n       'platelet_count', 'potassium', 'repiratory_rate', 'sodium',\n       'wbc', 'death_365_days']\n"

###Neighbourhood solution

In [ ]:

def get_neighbor_solution(solution):
    neighbor_solution = solution.copy()  # Create a copy of the current solution

    # Define the column sets for swapping
    swap_sets = {
        'age': [ 'L1_age', 'L2_age', 'L3_age', 'L4_age', 'L5_age', 'L6_age', 'L7_age'],
        'los_hours': ['L1_los_hours ', 'L2_los_hours ', 'L3_los_hours ', 'L4_los_hours ', 'L5_los_hours', 'L6_los_hours', 'L7_los_hours'],
        'admission_type': ['L2_Admission_Type', 'L1_Admission_Type'],
        'diseases': ['L1_Diseases', 'L2_Diseases'],
        'ethnicity': ['L1_Ethnicity', 'L2_Ethnicity', 'L3_Ethnicity']
    }

    # Iterate over each column set and perform swapping if the current column is from that set
    # Randomly select a set to swap columns
    selected_set = random.choice(list(swap_sets.keys()))
    print('selected set:', selected_set)

    # Randomly shuffle the columns within the selected set
    random.shuffle(swap_sets[selected_set])

    # Replace the corresponding columns in the current solution with the shuffled columns
    for idx, col in enumerate(solution):
        if col in swap_sets[selected_set]:
            neighbor_solution[idx] = random.choice(swap_sets[selected_set])
            print('current solution:', solution)
            print('neighbor solution:', neighbor_solution)

    return neighbor_solution

###Simulated Annealing

In [ ]:
#df.head(10).T

In [ ]:
import math
import random

def simulated_annealing(initial_solution, initial_temperature, cooling_rate, min_temperature, max_iterations_without_improvement, max_suppressed_fraction):
    current_solution = initial_solution  # Store the current solution
    best_solution = initial_solution  # Initialize the best solution as the current solution
    current_temperature = initial_temperature  # Start with the initial temperature
    iterations_without_improvement = 0  # Initialize counter for iterations without improvement
    iteration = 0  # Initialize the iteration counter


    while current_temperature > min_temperature and iterations_without_improvement < max_iterations_without_improvement:
        improved = False

        for _ in range(num_iterations):
            iteration += 1
            print(f"Iteration {iteration}")

            neighbor_solution = get_neighbor_solution(current_solution)

            current_cost = objective_function(current_solution, max_suppressed_fraction)
            print('current_cost:', current_cost)
            neighbor_cost = objective_function(neighbor_solution, max_suppressed_fraction)
            print('neighbor_cost:', neighbor_cost)


            if neighbor_cost >= current_cost:
                current_solution = neighbor_solution
                #if neighbor_cost < objective_function(best_solution, max_suppressed_fraction):
                if neighbor_cost > objective_function(best_solution, max_suppressed_fraction): #and current_cost <= max_suppressed_fraction :
                    best_solution = neighbor_solution
                    print('current best solution:', best_solution)
                    improved = True
            else:
                acceptance_probability = math.exp((current_cost - neighbor_cost) / current_temperature)
                if random.random() < acceptance_probability:
                    current_solution = neighbor_solution
                    improved = True

            print('CURRENT SOLUTION', current_solution)
            if iteration % num_iterations == 0:
                current_temperature *= cooling_rate
                iterations_without_improvement = 0

            if not improved:
                iterations_without_improvement += num_iterations

        current_temperature *= cooling_rate
        iterations_without_improvement += num_iterations

        if improved:
            iterations_without_improvement = 0
        else:
            iterations_without_improvement += num_iterations

    return best_solution


max_suppressed_fraction = 0.20
best_solution = simulated_annealing(initial_solution, initial_temperature, cooling_rate, min_temperature, max_iterations_without_improvement, max_suppressed_fraction)
#auc_roc = objective_function(best_solution, max_suppressed_fraction)  # Calculate the classification accuracy for the best solution



print("Best Solution:", best_solution)
#print("AUC-ROC score:", auc_roc)


Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
X columns: Index(['L1_age', 'L1_los_hours', 'L2_Admission_Type', 'gender', 'stroke_type',
       'marital_status', 'L2_Diseases', 'L2_Ethnicity', 'infarct_type',
       'anion_gap', 'abp_diastolic', 'abp_systolic', 'bicarbonate', 'calcium',
       'chloride', 'creatinine', 'gcs', 'glucose', 'heart_rate', 'hemoglobin',
       'oasis', 'wbc', 'platelet_count', 'potassium', 'repiratory_rate',
       'sodium'],
      dtype='object')
Number of rows in X: 2195
Number of rows in y: 2195
0:	learn: 0.6656622	total: 10.8ms	remaining: 1.61s
1:	learn: 0.6372840	total: 22.9ms	remaining: 1.69s
2:	learn: 0.6138585	total: 33.7ms	remaining: 1.65s
3:	learn: 0.5956295	total: 44.9ms	remaining: 1.64s
4:	learn: 0.5768672	total: 56.3ms	remaining: 1.63s
5:	learn: 0.5621761	total: 67.8ms	remaining: 1.63s
6:	learn: 0.5445236	total: 78.4ms	remaining: 1.6s
7:	learn: 0.5329335	total: 89.5ms	remaining: 1.59s
8:	learn: 0.5246508	total: 100ms	remainin

### Testing the correctness of K anonymity on the best_solution (near-optimal combination of generalized attributes) produced by the SA algorithm

In [ ]:
'''
# Define the privacy parameter
#k = 2

#For testing purposes
#test_solution = ['Level_6_Age', 'Level_6_los_hours', 'L2_Admission_Type', 'gender', 'stroke_type', 'marital_status', 'L3_Diseases', 'L4_Ethnicity']
#test_solution = ['L1_age', 'L1_los_hours', 'L1_Admission_Type', 'gender', 'stroke_type', 'marital_status', 'L1_Diseases', 'L1_Ethnicity']


test_solution = best_solution

df_k = df

# Group the dataset by the quasi-identifiers and suppress the groups with less than k rows
grouped = df_k.groupby(test_solution)
suppressed_indices = []
for group_name, group in grouped:
    if len(group) < k:
        suppressed_indices.extend(group.index)
df_k = df_k.drop(suppressed_indices)
#df_k.info()
# Restore the original index
df_index = df_k.index
df_k = df_k.reset_index(drop=True)

counter = 0
for index in suppressed_indices:
    counter = counter + 1
    #print(df_k.loc[index])
    #print("------------")
print('sum of supressed records:', counter)
total_rows = len(df)
suppressed_rows = counter
suppressed_fraction = suppressed_rows / total_rows
print('SUPPRESSION FRACTION', suppressed_fraction)
'''

'\n# Define the privacy parameter\n#k = 2\n\n#For testing purposes\n#test_solution = [\'Level_6_Age\', \'Level_6_los_hours\', \'L2_Admission_Type\', \'gender\', \'stroke_type\', \'marital_status\', \'L3_Diseases\', \'L4_Ethnicity\']\n#test_solution = [\'L1_age\', \'L1_los_hours\', \'L1_Admission_Type\', \'gender\', \'stroke_type\', \'marital_status\', \'L1_Diseases\', \'L1_Ethnicity\']\n\n\ntest_solution = best_solution\n\ndf_k = df\n\n# Group the dataset by the quasi-identifiers and suppress the groups with less than k rows\ngrouped = df_k.groupby(test_solution)\nsuppressed_indices = []\nfor group_name, group in grouped:\n    if len(group) < k:\n        suppressed_indices.extend(group.index)\ndf_k = df_k.drop(suppressed_indices)\n#df_k.info()\n# Restore the original index\ndf_index = df_k.index\ndf_k = df_k.reset_index(drop=True)\n\ncounter = 0\nfor index in suppressed_indices:\n    counter = counter + 1\n    #print(df_k.loc[index])\n    #print("------------")\nprint(\'sum of supresse

In [ ]:
#df.info()

### Accurate Classification output

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report

selected_columns = best_solution

#k=10

df_k = df
additional_columns = ['infarct_type', 'anion_gap', 'abp_diastolic',
       'abp_systolic', 'bicarbonate', 'calcium', 'chloride', 'creatinine',
        'gcs', 'glucose', 'heart_rate',
       'hemoglobin',  'oasis', 'wbc',
       'platelet_count', 'potassium', 'repiratory_rate', 'sodium',
       'death_365_days']

#features = ['L2_age', 'L1_los_hours', 'L1_Admission_Type', 'gender', 'stroke_type', 'marital_status', 'L1_Diseases', 'L1_Ethnicity']
features = best_solution
# Group the dataset by the quasi-identifiers and suppress the groups with less than k rows
grouped = df_k.groupby(features)
suppressed_indices = []
for group_name, group in grouped:
    if len(group) < k:
        suppressed_indices.extend(group.index)
df_k = df_k.drop(suppressed_indices)
df_k.info()
# Restore the original index
df_index = df_k.index
df_k = df_k.reset_index(drop=True)

featurez_matrix = df_k[features + additional_columns]

# Prepare the feature matrix (X) and target variable (y)
X = featurez_matrix.drop('death_365_days', axis=1)
y = featurez_matrix['death_365_days']


counter = 0
for index in suppressed_indices:
    counter = counter + 1
    #print(df_k.loc[index])
    #print("------------")
print('sum of supressed records:', counter)
total_rows = len(df)
suppressed_rows = counter
suppressed_fraction = suppressed_rows / total_rows
print('SUPPRESSION FRACTION', suppressed_fraction)
print('K-value:', k)
print('anon df columns:', df_k.columns)



# Convert categorical features to strings
categorical_features_indices = X.select_dtypes(include=['object']).columns
#X[categorical_features_indices] = X[categorical_features_indices].astype(str)
#seed = 23 for tests, seed= 28 for baseline
random_seed = 32
np.random.seed(random_seed)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)



# Initialize the CatBoostClassifier 350.9,0.08
catboost_model = CatBoostClassifier(iterations=350, depth=9, learning_rate=0.08, loss_function='Logloss', cat_features=list(categorical_features_indices))

# Train the model on the training data
catboost_model.fit(X_train, y_train, cat_features=list(categorical_features_indices))  # Convert to list here

# Predict on the testing data
y_pred = catboost_model.predict(X_test)
# Convert the predicted labels to string type
#y_pred = y_pred.astype(str)


# Evaluate the model's performance
classification_metrics = classification_report(y_test, y_pred, digits=3)
print(classification_metrics)


# Assuming you have y_test and y_pred with labels '<=50K' and '>50K'
# Convert labels to binary values (0 and 1)
#y_test_binary = np.where(y_test == '<=50K', 0, 1)
#y_pred_binary = np.where(y_pred == '<=50K', 0, 1)

# Calculate the AUC-ROC score
auc_roc = roc_auc_score(y_test, y_pred)
print("AUC-ROC Score: {:.3f}".format(auc_roc))


counter = 0
for index in suppressed_indices:
    counter = counter + 1
    #print(df_k.loc[index])
    #print("------------")
print('sum of supressed records:', counter)
total_rows = len(df)
suppressed_rows = counter
suppressed_fraction = suppressed_rows / total_rows
print('SUPPRESSION FRACTION', suppressed_fraction)
print('K-value:', k)
print('anon df columns:', X.columns)
print('best solution', best_solution)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2253 entries, 0 to 2654
Data columns (total 44 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   L7_age             2253 non-null   float64
 1   gender             2253 non-null   object 
 2   L3_Ethnicity       2253 non-null   object 
 3   marital_status     2253 non-null   object 
 4   infarct_type       2253 non-null   object 
 5   anion_gap          2253 non-null   float64
 6   abp_diastolic      2253 non-null   float64
 7   abp_systolic       2253 non-null   float64
 8   bicarbonate        2120 non-null   float64
 9   calcium            2253 non-null   float64
 10  chloride           2253 non-null   float64
 11  creatinine         2253 non-null   float64
 12  gcs                2240 non-null   float64
 13  glucose            2253 non-null   float64
 14  heart_rate         2253 non-null   float64
 15  hemoglobin         2253 non-null   float64
 16  oasis              2253 

In [ ]:
'''
features = ['age', 'gender', 'L4_Ethnicity', #'weight',
       'infarct_type', 'anion_gap', 'abp_diastolic', 'marital_status',
       'abp_systolic', 'bicarbonate', 'calcium', 'chloride', 'creatinine',
        'gcs', 'glucose', 'heart_rate',
       'hemoglobin',  'oasis',
       'los_hours', 'platelet_count', 'potassium', 'repiratory_rate', 'sodium',
       'wbc', 'stroke_type', 'L2_Admission_Type', 'L3_Diseases',
       'death_365_days']
'''

In [ ]:
stop

In [ ]:
X.info()

### Baseline classification score

In [ ]:
df.info()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
from sklearn.impute import SimpleImputer
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score, classification_report

#selected_columns = best_solution

# Additional columns to include in the feature set
# maybe add oasis and glucose later

'''
features = ['age', 'marital_status', 'diseases', 'ethnicity', 'admission_type', 'los_hours', 'stroke_type', 'abp_diastolic',
       'abp_systolic', 'creatinine', 'anion_gap', 'sodium', 'potassium',
        'heart_rate', 'hemoglobin', 'oasis', 'glucose', 'calcium',
                       'death_365_days']
'''

features = ['infarct_type', 'anion_gap', 'abp_diastolic', 'marital_status',
       'abp_systolic', 'bicarbonate', 'calcium', 'chloride', 'creatinine',
        'gcs', 'glucose', 'heart_rate',
       'hemoglobin',  'oasis', 'wbc',
       'platelet_count', 'potassium', 'repiratory_rate', 'sodium',
       'death_365_days',
            'L7_age', 'gender', 'L3_Ethnicity', 'L2_Admission_Type', 'L2_Diseases', 'L7_los_hours']


featurez_matrix = df[features]
# Set a random seed for reproducibility
random_seed = 32  # You can use any integer value as the seed
np.random.seed(random_seed)

# Prepare the feature matrix (X) and target variable (y)
X = featurez_matrix.drop('death_365_days', axis=1)
y = featurez_matrix['death_365_days']


# Convert categorical features to strings
categorical_features_indices = X.select_dtypes(include=['object']).columns
#X[categorical_features_indices] = X[categorical_features_indices].astype(str)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_seed)



# Initialize the CatBoostClassifier #depth 11, iterations 600 le .04 ergab auc 0.793
catboost_model = CatBoostClassifier(iterations=350, depth=9, learning_rate=0.08, loss_function='Logloss', cat_features=list(categorical_features_indices))

# Train the model on the training data
catboost_model.fit(X_train, y_train, cat_features=list(categorical_features_indices))  # Convert to list here

# Predict on the testing data
y_pred = catboost_model.predict(X_test)
# Convert the predicted labels to string type
y_pred = y_pred.astype(str)


# Evaluate the model's performance
classification_metrics = classification_report(y_test, y_pred)
print(classification_metrics)

# Calculate AUC-ROC score

auc_roc = roc_auc_score(y_test.astype(int), y_pred.astype(int))
print("AUC-ROC Score:", auc_roc)



In [ ]:
print("Categorical Feature Indices:", categorical_features_indices)


In [ ]:
print("Best Solution:", best_solution)
print("Optimization score:", suppressed_rows)

In [ ]:
import matplotlib.pyplot as plt

# Get feature importances
feature_importances = catboost_model.feature_importances_

# Get feature names
feature_names = X.columns

# Sort feature importances in descending order
sorted_idx = np.argsort(feature_importances)[::-1]

# Plot feature importances
plt.figure(figsize=(20, 6))
plt.bar(range(len(feature_names)), feature_importances[sorted_idx])
plt.xticks(range(len(feature_names)), np.array(feature_names)[sorted_idx], rotation='vertical')
plt.xlabel("Feature")
plt.ylabel("Feature Importance")
plt.title("Feature Importances")


# Annotate feature importances next to the bars
for i, idx in enumerate(sorted_idx):
    plt.text(i, feature_importances[idx] + 0.01, f"{feature_importances[idx]:.3f}", ha='center')

plt.tight_layout()
plt.show()

In [ ]:
X.info()

The interaction between the average generalization level and the suppressed fraction can lead to sudden changes in the combined score. Small changes in the average generalization level might lead to a larger effect on the suppressed fraction, causing significant changes in the combined score.

OR

The optimization function uses a weight parameter to balance between the two objectives (suppressed fraction and average generalization). The sudden change in the number of suppressed records might be due to the sensitivity of the function to the specific weight value I'm using.

The following are 2 different objective functions that measure only the number of supressed rows. They were in the very first version in this project but expanded to include generalization degree later. They simply implement k-anonymity.

In [ ]:
'''
# Define the objective function
def objective_function(solution):
    suppressed_rows = 0  # Counter for suppressed rows
    groups = {}  # Dictionary to group rows based on quasi identifiers
    for index, row in dataset.iterrows():  # Iterate over each row in the dataset
        key = tuple(row[quasi_identifier] for quasi_identifier in solution)  # Create a key tuple based on the values in the current row for the given solution
        if key in groups:  # If the key already exists in groups, add the row to the existing group
            groups[key].append(row)
        else:  # If the key doesn't exist, create a new group with the current row
            groups[key] = [row]
    for group in groups.values():  # Iterate over each group
        group_size = len(group)  # Get the size of the group
        if group_size < k:  # If the group size is less than the k-anonymity threshold
            suppressed_rows += group_size  # Add the group size to the suppressed rows count
    return suppressed_rows  # Return the total number of suppressed rows

'''

''' #simple but works well
def objective_function(solution):
    grouped = df.groupby([df[qi] for qi in solution])
    suppressed_indices = []
    for group_name, group in grouped:
        if len(group) < k:
            suppressed_indices.extend(group.index)
    suppressed_count = len(suppressed_indices)
    return suppressed_count
'''
